YOLO Code

In [1]:
!pip install nets
!pip install --upgrade ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.2/71.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 903.1/903.1 kB 23.3 MB/s eta 0:00:00


In [2]:
!pip install cupy-cuda11x

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 MB 8.2 MB/s eta 0:00:00


In [3]:
import os
import zipfile
import shutil
import cv2
import torch
from ultralytics import YOLO
import csv
from datetime import datetime

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
def extract_dataset(zip_path, extract_to="dataset"):
    """Extracts a ZIP file to the specified directory."""
    if os.path.exists(extract_to):
        shutil.rmtree(extract_to)
    os.makedirs(extract_to, exist_ok=True)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

    print(f"Dataset extracted to {extract_to}")
    return extract_to

In [5]:
def prepare_yolo_dataset(dataset_path):
    """Prepares the YOLO dataset from extracted files."""
    images_dir = os.path.join(dataset_path, "images")
    labels_dir = os.path.join(dataset_path, "labels")
    os.makedirs(images_dir, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)

    for student_folder in os.listdir(dataset_path):
        student_path = os.path.join(dataset_path, student_folder)
        if not os.path.isdir(student_path):
            continue

        regno = student_folder  # Registration number is the class
        for img_file in os.listdir(student_path):
            if img_file.endswith(('.jpg', '.png', '.jpeg')):
                # Move images to the images directory
                src_img = os.path.join(student_path, img_file)
                dst_img = os.path.join(images_dir, f"{regno}_{img_file}")
                shutil.move(src_img, dst_img)

                # Assume labels are in YOLO format in a `.txt` file
                label_file = os.path.splitext(img_file)[0] + ".txt"
                src_label = os.path.join(student_path, label_file)
                if os.path.exists(src_label):
                    dst_label = os.path.join(labels_dir, f"{regno}_{label_file}")
                    shutil.move(src_label, dst_label)

    print("YOLO dataset prepared.")
    return images_dir, labels_dir

In [6]:
def train_yolo_model(data_config):
    """Train YOLO model using provided configuration."""
    try:
        model = YOLO('yolov8n.pt')  # Use valid weights
        model.train(data=data_config, epochs=250, imgsz=640)
        print("Model training complete.")
        return model
    except AttributeError as e:
        print(f"AttributeError: {e}")
        print("Ensure the ultralytics library is correctly installed and YOLO is imported.")
        raise
    except Exception as e:
        print(f"An error occurred: {e}")
        raise

In [7]:
def detect_students(model, group_image_path):
    """Detect students in a group image."""
    results = model.predict(source=group_image_path, save=True)
    print("Detection complete. Results saved.")

    detected_students = []
    for result in results:
        if result.boxes is not None:
            detected_students.extend([int(box.cls) for box in result.boxes])

    return detected_students

In [8]:
def mark_attendance(detected_students, attendance_file="attendance.csv", class_name="Class A"):
    """Marks attendance of detected students."""
    current_date = datetime.now().strftime("%Y-%m-%d")
    current_day = datetime.now().strftime("%A")

    file_exists = os.path.isfile(attendance_file)

    with open(attendance_file, mode='a', newline='') as csvfile:
        fieldnames = ['Date', 'Day', 'Class', 'Registration Number']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        if not file_exists:
            writer.writeheader()

        for regno in detected_students:
            writer.writerow({
                'Date': current_date,
                'Day': current_day,
                'Class': class_name,
                'Registration Number': regno
            })

    print(f"Attendance marked for {len(detected_students)} students in {attendance_file}.")

In [9]:
zip_path = "/content/roboflow.zip"
group_image_path = "/content/WhatsApp Image 2024-12-19 at 9.28.48 PM.jpeg"
attendance_file = "/content/drive/MyDrive/attendance.csv"

dataset_path = extract_dataset(zip_path)
images_dir, labels_dir = prepare_yolo_dataset(dataset_path)

Dataset extracted to dataset
YOLO dataset prepared.


In [10]:
import yaml


# Save the data configuration as a YAML file
yaml_file = "/content/data.yaml"

def train_yolo_model(yaml_path):
    try:
        # Load the YOLO model
        model = YOLO("yolov8n.pt")  # Ensure the weights file exists
        model.train(data=yaml_path, epochs=250, imgsz=640)
        print("Model training complete.")
        return model
    except Exception as e:
        print(f"An error occurred during training: {e}")
        raise

# Pass the path to the YAML file
model = train_yolo_model(yaml_file)


100%|██████████| 6.25M/6.25M [00:00<00:00, 114MB/s]


Ultralytics 8.3.54 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/data.yaml, epochs=250, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, 

100%|██████████| 755k/755k [00:00<00:00, 22.3MB/s]


Overriding model.yaml nc=80 with nc=9

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 86.4MB/s]


...

    250/250      1.63G     0.2687      1.197     0.7896         10        640: 100%|██████████| 1/1 [00:00<00:00,  4.73it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.55it/s]

                   all          4          4      0.985        0.5      0.995      0.729



250 epochs completed in 0.074 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.3MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.54 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,007,403 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 10.82it/s]


                   all          4          4      0.985        0.5      0.995      0.729
               2022002          3          3      0.971          1      0.995      0.464
               2022029          1          1          1          0      0.995      0.995
Speed: 0.3ms preprocess, 5.1ms inference, 0.0ms loss, 2.9ms postprocess per image
Results saved to runs/detect/train
Model training complete.


In [12]:
def detect_and_identify_person(model, image_path, class_labels, output_dir="detections"):
    os.makedirs(output_dir, exist_ok=True)

    # Perform detection
    results = model.predict(source=image_path, save=True, save_txt=True, save_conf=True, project=output_dir)

    # Initialize detected students
    detected_students = []

    for result in results:
        if result.boxes is not None:
            for box in result.boxes:
                class_id = int(box.cls)  # Class ID
                confidence = box.conf  # Confidence score
                detected_students.append(class_labels[class_id])
                print(f"Detected: {class_labels[class_id]} with confidence {confidence:.2f}")

    print("Detection complete. Check the results directory for saved outputs.")
    return detected_students


# Example usage
group_image_path = "/content/WhatsApp Image 2024-12-19 at 8.11.34 PM (1).jpeg"
class_labels = [ '2022002', '2022029','2022038', '2022074', '2022076', '2022081','2022088', '2022101', '2022120','2022133','2022137','2022201','2022214','2022246','2022248',
  '2022909', '2022267', '2022274','2022275', '2022277', '2022284', '2022907', '2022295','2022313','2022378','2022383','2022387','2022405','2022414','2022477','2022488','2022502','2022531', '2022559','2022560','2022568','2022622', '2022623','2022624', '2022655','2022656', '2022660','2022664', '2022669', '2022672'

]

detected_students = detect_and_identify_person(model, group_image_path, class_labels)

# Mark attendance for the detected students
mark_attendance(detected_students, attendance_file)



image 1/1 /content/WhatsApp Image 2024-12-19 at 8.11.34 PM (1).jpeg: 640x480 (no detections), 37.0ms
Speed: 2.8ms preprocess, 37.0ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 480)
Results saved to detections/train2
0 label saved to detections/train2/labels
Detection complete. Check the results directory for saved outputs.
Attendance marked for 0 students in /content/drive/MyDrive/attendance.csv.
